# Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation

This notebook implements the model in:  
Cho, K., Van Merriënboer, B., Gulcehre, C., Bahdanau, D., Bougares, F., Schwenk, H., and Bengio, Y. 2014. Learning phrase representations using RNN encoder-decoder for statistical machine translation. arXiv preprint [arXiv:1406.1078](https://arxiv.org/abs/1406.1078).  

This model is based on an *Encoder-Decoder* framework, in which the encoder and the decoder are both RNNs.  
This model encodes all the information of the source sequence into a fixed-length context vector $z$, and then utilizes $z$ as an input in *every step* when generating the target sequence, instead of utilizing $z$ only in the beginning of generation. This design aims to relieve the *information compression*.  

![Learning Phrase Representations](fig/learning-phrase-representations.png)

In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

SEED = 515
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## Preparing Data

In [2]:
import spacy
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

def tokenize_de(text):
    """
    Tokenize German text. 
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenize English text.
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [3]:
from torchtext.data import Field, BucketIterator

SRC = Field(tokenize=tokenize_de, init_token='<sos>', eos_token='<eos>', 
            lower=True, include_lengths=True)
TRG = Field(tokenize=tokenize_en, init_token='<sos>', eos_token='<eos>', 
            lower=True, include_lengths=True)

In [4]:
from torchtext.datasets import Multi30k

train_data, valid_data, test_data = Multi30k.splits(exts=['.de', '.en'], 
                                                    # fields=[SRC, TRG], 
                                                    fields=[('src', SRC), ('trg', TRG)], 
                                                    root='data/')

In [5]:
print(train_data[0].src)
print(train_data[0].trg)

['zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '.']
['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']


In [6]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

len(SRC.vocab), len(TRG.vocab)

(7855, 5893)

In [7]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE, device=device)

In [8]:
for batch in train_iterator:
    batch_src, batch_src_lens = batch.src
    batch_trg, batch_trg_lens = batch.trg
    break
print(batch_src)
print(batch_src_lens)
print(batch_trg)
print(batch_trg_lens)

tensor([[  2,   2,   2,  ...,   2,   2,   2],
        [  5,   5,  43,  ...,   5,  18,  18],
        [ 13,  13, 253,  ...,  13,  30,   0],
        ...,
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1]], device='cuda:0')
tensor([14, 17, 12, 11, 17, 21, 12, 16, 14, 11, 23, 23,  8, 11,  9, 14, 19, 20,
        12, 16,  9, 11, 13, 20, 21, 29, 13, 22, 14, 16, 10,  9, 15, 12, 17, 10,
        14, 22, 17, 20, 23, 23, 12, 17, 15, 19, 17, 15, 16,  7, 14, 15, 16, 12,
        17, 14, 18, 18, 14, 14, 17, 21, 12, 12,  9, 19, 12, 14, 12, 11, 10, 13,
        18, 14,  9, 11, 10, 12, 10, 25, 14, 18, 15, 16, 15, 18, 13,  9, 21, 11,
        20, 12, 13, 14, 14, 17, 10, 13, 18, 30, 14, 12, 13,  9, 10, 15, 13, 10,
        12, 15, 13, 18, 17, 13, 11, 12, 10, 16, 12, 13, 24, 14, 19, 19, 10, 20,
        12, 11], device='cuda:0')
tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [   4,    4,   48,  ...,    4,   16,   

## Build Model
### Encoder
* Use GRU instead of LSTM.  

In [9]:
class Encoder(nn.Module):
    def __init__(self, in_dim, emb_dim, hid_dim, n_layers, dropout, pad_idx):
        super().__init__()
        self.emb = nn.Embedding(in_dim, emb_dim, padding_idx=pad_idx)
        self.rnn = nn.GRU(emb_dim, hid_dim, num_layers=n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_lens):
        # src: (step, batch)
        embedded = self.dropout(self.emb(src))

        # Pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, src_lens, enforce_sorted=False)
        # hidden: (num_layers*num_directions, batch, hid_dim)
        packed_outs, hidden = self.rnn(packed_embedded)
        return hidden

In [10]:
SRC_IN_DIM = len(SRC.vocab)
TRG_IN_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
ENC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
DEC_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

encoder = Encoder(SRC_IN_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT, ENC_PAD_IDX).to(device)
hidden = encoder(batch_src, batch_src_lens)

print(batch_src.size())
print(hidden.size())

torch.Size([30, 128])
torch.Size([2, 128, 512])


### Decoder
* Use GRU instead of LSTM.  
* Combine the `embedding` and the `context` as the input to the RNN.  
* Combine the `embedding`, the (last layer) `hidden` and `context` as the input to the output FC. 

In [11]:
class Decoder(nn.Module):
    def __init__(self, in_dim, emb_dim, hid_dim, n_layers, dropout, pad_idx):
        super().__init__()
        self.emb = nn.Embedding(in_dim, emb_dim, padding_idx=pad_idx)
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim, num_layers=n_layers, dropout=dropout)
        # The output dimension equals the input dimension for the decoder.
        self.fc = nn.Linear(emb_dim + hid_dim*2, in_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, trg, hidden, context):
        """
        One-step forward. 
        """
        # trg: (step=1, batch)
        # context: (step=1, batch, hid_dim)
        embedded = self.dropout(self.emb(trg))

        # outs: (step=1, batch, hid_dim)
        # hidden: (num_layers*num_directions, batch, hid_dim)
        outs, hidden = self.rnn(torch.cat([embedded, context], dim=-1), 
                                hidden)
        # preds: (step=1, batch, out_dim=in_dim)
        preds = self.fc(torch.cat([embedded, outs, context], dim=-1))
        return preds, hidden

In [12]:
decoder = Decoder(TRG_IN_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT, DEC_PAD_IDX).to(device)

context = hidden[-1].unsqueeze(0)
preds, hidden = decoder(batch_trg[0].unsqueeze(0), hidden, context)

print(context.size())
print(batch_trg.size())
print(preds.size())
print(hidden.size())

torch.Size([1, 128, 512])
torch.Size([29, 128])
torch.Size([1, 128, 5893])
torch.Size([2, 128, 512])


### Seq2Seq

In [13]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, src_lens, trg, teacher_forcing_ratio=0.5):
        # src: (step, batch)
        # trg: (step, batch)
        hidden = self.encoder(src, src_lens)
        
        # context: (step=1, batch, hid_dim)
        # context is the last hidden state on the top layer. 
        context = hidden[-1].unsqueeze(0)

        preds = []
        # The first input to the decoder is the <sos> token. 
        # trg_t: (step=1, batch)
        trg_t = trg[0].unsqueeze(0)
        for t in range(1, trg.size(0)):
            # preds_t: (step=1, batch, trg_out_dim)
            preds_t, hidden = self.decoder(trg_t, hidden, context)
            # top1: (step=1, batch)
            top1 = preds_t.argmax(dim=-1)
            if np.random.rand() < teacher_forcing_ratio:
                trg_t = trg[t].unsqueeze(0)
            else:
                trg_t = top1
            preds.append(preds_t)
        # preds: (step-1, batch, trg_out_dim)
        return torch.cat(preds, dim=0)

In [14]:
model = Seq2Seq(encoder, decoder).to(device)
preds = model(batch_src, batch_src_lens, batch_trg)

print(batch_trg.size())
print(preds.size())

torch.Size([29, 128])
torch.Size([28, 128, 5893])


## Train Model

In [15]:
def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)

def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

N_LAYERS = 1
encoder = Encoder(SRC_IN_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT, ENC_PAD_IDX)
decoder = Decoder(TRG_IN_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT, DEC_PAD_IDX)
model = Seq2Seq(encoder, decoder).to(device)

model.apply(init_weights)
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 14,220,293 trainable parameters


In [16]:
# Initialize Embeddings 
ENC_UNK_IDX = SRC.vocab.stoi[SRC.unk_token]
DEC_UNK_IDX = TRG.vocab.stoi[TRG.unk_token]

model.encoder.emb.weight.data[ENC_UNK_IDX].zero_()
model.encoder.emb.weight.data[ENC_PAD_IDX].zero_()
model.decoder.emb.weight.data[DEC_UNK_IDX].zero_()
model.decoder.emb.weight.data[DEC_PAD_IDX].zero_()

print(model.encoder.emb.weight[:5, :8])
print(model.decoder.emb.weight[:5, :8])

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0171,  0.0020, -0.0057,  0.0012, -0.0194,  0.0050, -0.0010, -0.0051],
        [-0.0062, -0.0034, -0.0117, -0.0156, -0.0005, -0.0193, -0.0133, -0.0189],
        [ 0.0105, -0.0096,  0.0139,  0.0010,  0.0069, -0.0085, -0.0098, -0.0005]],
       device='cuda:0', grad_fn=<SliceBackward>)
tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-2.1665e-02, -1.0329e-02, -1.3528e-03, -1.2173e-02, -9.7946e-05,
         -8.4786e-03,  1.6576e-03,  5.9450e-03],
        [ 4.2984e-03,  1.3037e-02, -3.5510e-03, -6.2088e-03,  2.2002e-03,
         -1.5345e-02,  1.8932e-03,  1.2644e-03],
        [-1.3398e-02,  1.2008e-02,  1.3676e-02, 

In [17]:
loss_func = nn.CrossEntropyLoss(ignore_index=DEC_PAD_IDX, reduction='mean')
optimizer = optim.AdamW(model.parameters())

We must ensure we turn `teacher forcing` off for evaluation. This will cause the model to only use it's own predictions to make further predictions within a sentence, which mirrors how it would be used in deployment. 

In [18]:
def train_epoch(model, iterator, optimizer, loss_func, clip):
    model.train()
    epoch_loss = 0
    for batch in iterator:
        # Forward pass
        batch_src, batch_src_lens = batch.src
        batch_trg, batch_trg_lens = batch.trg
        # preds: (step-1, batch, trg_out_dim)
        preds = model(batch_src, batch_src_lens, batch_trg)
        
        # Calculate loss
        preds_flattened = preds.view(-1, preds.size(-1))
        batch_trg_flattened = batch_trg[1:].flatten()
        loss = loss_func(preds_flattened, batch_trg_flattened)

        # Backward propagation
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)

        # Update weights
        optimizer.step()
        # Accumulate loss
        epoch_loss += loss.item()
    return epoch_loss/len(iterator)

def eval_epoch(model, iterator, loss_func):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for batch in iterator:
            # Forward pass
            batch_src, batch_src_lens = batch.src
            batch_trg, batch_trg_lens = batch.trg
            # preds: (step-1, batch, trg_out_dim)
            preds = model(batch_src, batch_src_lens, batch_trg, teacher_forcing_ratio=0)
            
            # Calculate loss
            preds_flattened = preds.view(-1, preds.size(-1))
            batch_trg_flattened = batch_trg[1:].flatten()
            loss = loss_func(preds_flattened, batch_trg_flattened)
            
            # Accumulate loss and acc
            epoch_loss += loss.item()
    return epoch_loss/len(iterator)

In [19]:
import time
N_EPOCHS = 10
CLIP = 1
best_valid_loss = np.inf

for epoch in range(N_EPOCHS):
    t0 = time.time()
    train_loss = train_epoch(model, train_iterator, optimizer, loss_func, CLIP)
    valid_loss = eval_epoch(model, valid_iterator, loss_func)
    epoch_secs = time.time() - t0

    epoch_mins, epoch_secs = int(epoch_secs // 60), int(epoch_secs % 60)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'models/tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {np.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {np.exp(valid_loss):7.3f}')

Epoch: 01 | Epoch Time: 0m 15s
	Train Loss: 5.084 | Train PPL: 161.362
	 Val. Loss: 4.884 |  Val. PPL: 132.102
Epoch: 02 | Epoch Time: 0m 14s
	Train Loss: 4.365 | Train PPL:  78.626
	 Val. Loss: 4.638 |  Val. PPL: 103.323
Epoch: 03 | Epoch Time: 0m 14s
	Train Loss: 3.989 | Train PPL:  54.012
	 Val. Loss: 4.338 |  Val. PPL:  76.519
Epoch: 04 | Epoch Time: 0m 13s
	Train Loss: 3.636 | Train PPL:  37.937
	 Val. Loss: 4.141 |  Val. PPL:  62.847
Epoch: 05 | Epoch Time: 0m 13s
	Train Loss: 3.319 | Train PPL:  27.621
	 Val. Loss: 3.882 |  Val. PPL:  48.500
Epoch: 06 | Epoch Time: 0m 14s
	Train Loss: 3.031 | Train PPL:  20.713
	 Val. Loss: 3.685 |  Val. PPL:  39.857
Epoch: 07 | Epoch Time: 0m 14s
	Train Loss: 2.762 | Train PPL:  15.827
	 Val. Loss: 3.627 |  Val. PPL:  37.598
Epoch: 08 | Epoch Time: 0m 14s
	Train Loss: 2.524 | Train PPL:  12.473
	 Val. Loss: 3.572 |  Val. PPL:  35.589
Epoch: 09 | Epoch Time: 0m 14s
	Train Loss: 2.307 | Train PPL:  10.044
	 Val. Loss: 3.587 |  Val. PPL:  36.126
E

In [20]:
model.load_state_dict(torch.load('models/tut2-model.pt'))

valid_loss = eval_epoch(model, valid_iterator, loss_func)
test_loss = eval_epoch(model, test_iterator, loss_func)

print(f'Val. Loss: {valid_loss:.3f} |  Val. PPL: {np.exp(valid_loss):7.3f}')
print(f'Test Loss: {test_loss:.3f} |  Test PPL: {np.exp(test_loss):7.3f}')

Val. Loss: 3.561 |  Val. PPL:  35.209
Test Loss: 3.514 |  Test PPL:  33.598


## Check Embeddings
* The Embeddings of `unk` and `<pad>` tokens
    * Because the `padding_idx` has been passed to `nn.Embedding`, so the `<pad>` embedding will remain zeros throughout training.  
    * While the `<unk>` embedding will be learned.

In [21]:
print(model.encoder.emb.weight[:5, :8])
print(model.decoder.emb.weight[:5, :8])

tensor([[-0.0038, -0.0012, -0.0009, -0.0096, -0.0163,  0.0270,  0.0042,  0.0362],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.0197, -0.0568,  0.0122, -0.1237,  0.0502,  0.0278,  0.0169],
        [-0.0323, -0.0191, -0.3063, -0.0024, -0.1557, -0.0716, -0.0217, -0.0214],
        [-0.0147, -0.0006, -0.2653, -0.0190, -0.1412, -0.0908, -0.0126, -0.0507]],
       device='cuda:0', grad_fn=<SliceBackward>)
tensor([[ 0.0108, -0.0334, -0.0061,  0.0508,  0.0139,  0.0177,  0.0347,  0.0121],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.1713,  0.0946, -0.0183, -0.1468,  0.1339, -0.0602, -0.1696, -0.0499],
        [ 0.0390,  0.0137,  0.0507,  0.0408,  0.0215, -0.0705, -0.0421, -0.0374],
        [-0.0880,  0.0909,  0.0851,  0.0224,  0.0130, -0.1222,  0.0286, -0.1645]],
       device='cuda:0', grad_fn=<SliceBackward>)
